Importação das Bibliotecas

In [8]:
import pysolr
import json
import numpy as np
import logging

Carregar variáveis globais

In [ ]:
%store -r id_consulta

Configuração do Logger

In [9]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

Carregando .jsons

In [10]:
with open('map_documentos.json', 'r', encoding='utf-8') as f:
    map_documentos = json.load(f)
with open('map_consultas.json', 'r', encoding='utf-8') as f:
    map_consultas = json.load(f)

Configuração da Conexão com o Solr

In [11]:
print("Conectando ao Solr...")
SOLR_URL_BASE = 'http://localhost:8983/solr/'
SOLR_CORE_NAME = 'documentos_ri'
SOLR_CONNECTION_STRING = f'{SOLR_URL_BASE}{SOLR_CORE_NAME}'

try:
    solr = pysolr.Solr(SOLR_CONNECTION_STRING, always_commit=True)
    solr.ping()
    logging.info(f"Conexão com o core '{SOLR_CORE_NAME}' bem-sucedida!")
except Exception as e:
    logging.error("Não foi possível conectar ao Solr. Verifique se o serviço está ativo.")
    logging.error(f"Detalhe: {e}")
    exit()


2025-07-22 08:09:47 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/admin/ping/?' (get) with body '' in 0.034 seconds, with status 200
2025-07-22 08:09:47 - INFO - Conexão com o core 'documentos_ri' bem-sucedida!


Conectando ao Solr...


Função que retorna Top N documentos de uma consulta no Solr

In [12]:
def obter_top_n_ids_solr(solr_client, consulta_tokenizada, n):
    # 1. Reconstrói a string de busca a partir dos tokens
    texto_consulta = " ".join(consulta_tokenizada)
    query_string = f'texto_documento:({texto_consulta})'

    # 2. Executa a busca no Solr, que já retorna os resultados ordenados
    resultados_solr = solr_client.search(q=query_string, fl='id', rows=n)

    # 3. Retorna apenas os 'n' primeiros IDs da lista ordenada
    # Converte o ID (string) para inteiro
    return [int(resultado['id']) for resultado in resultados_solr]


Executar consulta e gerar os resultados top 50

In [ ]:
logging.info("Executando consulta de exemplo...")
# Escolha uma consulta para testar
consulta_texto = map_consultas[id_consulta]
consulta_tokenizada = consulta_texto.split(" ")
# Tokeniza a consulta
consulta_tokenizada = texto_consulta_exemplo.lower().split()
# Top 50 documentos mais relevantes para a consulta
resultados_solr = obter_top_n_ids_solr(solr, consulta_tokenizada, 50)
logging.info(f"Encontrados {len(resultados_solr)} resultados para a consulta {id_consulta_exemplo}.")


2025-07-22 08:09:47 - INFO - Executando consulta de exemplo...
2025-07-22 08:09:47 - INFO - Finished 'http://localhost:8983/solr/documentos_ri/select/?q=texto_documento%3A%28role+apc+adenomatous+polyposis+coli+colon+cancer%29&fl=id&rows=50&wt=json' (get) with body '' in 0.027 seconds, with status 200
2025-07-22 08:09:47 - INFO - Encontrados 50 resultados para a consulta 3.


Salvando resultados_solr como .json

In [14]:
try:
    with open('resultados_solr.json', 'w', encoding='utf-8') as f:
        json.dump(
            resultados_solr,
            f,
            indent=2,
            ensure_ascii=False
        )
    logging.info("Arquivo de resultados do Solr salvo com sucesso!")
except Exception as e:
    logging.error(f"ERRO ao salvar o arquivo JSON: {e}")
    

2025-07-22 08:09:47 - INFO - Arquivo de resultados do Solr salvo com sucesso!
